In [80]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
import pandas as pd
df = pd.read_csv('Disinformation_Dozen_Activities.csv')

In [3]:
df.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Wed May 20 19:04:38 +0000 2020,Covid,NaN,https://www.charityauctionstoday.com/auctions/...,87,1263183873594347520,NaN,NaN,NaN,en,NaN,False,NaN,25,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",There are so many people struggling with #Covi...,https://twitter.com/RobertKennedyJr/status/126...,Mon Jul 18 16:22:27 +0000 2011,337808606,False,"Robert F. Kennedy, Jr. is founder, chairman + ...",116,469710,646,2293,"Los Angles, California",Robert F. Kennedy Jr,RobertKennedyJr,5302,NaN,https://childrenshealthdefense.org/defender/,True
1,NaN,Wed May 13 16:01:18 +0000 2020,NaN,NaN,http://ow.ly/j9KO50zEgZb,29,1260601020142489601,NaN,NaN,NaN,en,NaN,False,NaN,21,NaN,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...",Researchers at several universities and medica...,https://twitter.com/mercola/status/12606010201...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False
2,NaN,Sun May 31 15:10:25 +0000 2020,NaN,NaN,NaN,0,1267111194164412417,NaN,NaN,NaN,en,NaN,NaN,NaN,12,1.266743e+18,ClimateEFund,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Yes! ""Right now, real leaders should be doubli...",https://twitter.com/RobertKennedyJr/status/126...,Mon Jul 18 16:22:27 +0000 2011,337808606,False,"Robert F. Kennedy, Jr. is founder, chairman + ...",116,469710,646,2293,"Los Angles, California",Robert F. Kennedy Jr,RobertKennedyJr,5302,NaN,https://childrenshealthdefense.org/defender/,True
3,NaN,Sun May 10 04:06:23 +0000 2020,NaN,NaN,NaN,0,1259333939589775362,NaN,NaN,NaN,en,NaN,NaN,NaN,7756,1.258126e+18,delbigtree,"<a href=""http://twitter.com/download/iphone"" r...",WHAT WOULD YOU DO if your six year old son or ...,https://twitter.com/DrChrisNorthrup/status/125...,Tue Feb 16 17:33:01 +0000 2010,114807372,False,Dr. Christiane Northrup is a leading authority...,75,114775,594,1704,Maine,Christiane Northrup,DrChrisNorthrup,12072,NaN,http://www.drnorthrup.com,False
4,NaN,Sat Feb 08 00:00:54 +0000 2020,NaN,NaN,http://ow.ly/SrBE50yfE5K,61,1225932480819138561,NaN,NaN,NaN,en,NaN,False,NaN,33,NaN,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...",Loneliness is at epidemic proportions in the U...,https://twitter.com/mercola/status/12259324808...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False


In [4]:
# Get the recipients and remove the nan from the beginning for blanks
recipients = df["in_reply_to_screen_name"].unique()[1:]
recipients.sort()
recipients.size

306

In [5]:
# Get the recipients and remove the nan from the beginning for blanks
half_dozen = df["user_screen_name"].unique()
half_dozen.sort()
half_dozen

array(['DrButtar', 'DrChrisNorthrup', 'RobertKennedyJr', 'kevdjenkins1',
       'mercola', 'sayerjigmi'], dtype=object)

In [6]:
half_dozen_dict = {}
for dude in half_dozen:
    dude_recipients = df.loc[df['user_screen_name'] == dude]
    dude_recipients = dude_recipients['in_reply_to_screen_name'].unique()[1:]
    dude_recipients.sort()
    half_dozen_dict[dude] = dude_recipients

In [7]:
half_dozen_dict

{'DrButtar': array(['2600hz', '7dancingrain7', 'Achilles12219', 'AldnoahVoid',
        'AzamHussainUK', 'BayouGirl54', 'BelleElle82', 'BrittElise727',
        'CJforeveryoung', 'CarolM39', 'Corbynsmom7', 'DRockxxxx',
        'DarlaElaine0406', 'DollarrrrJim', 'DonCarlitos', 'DrButtar',
        'Gary133q', 'Gerrieag', 'Googoolygaga', 'HerpnutPat', 'JacquiJ13',
        'JimGlen3', 'JimW1967', 'Jmanuel7101', 'JonKeel',
        'JonMoor62699896', 'JoshAtDrButtar', 'JumbleWumbles',
        'KayBeck03904159', 'Kellie23364945', 'LairdDourka',
        'LindaForTrump1', 'MaryamHenein', 'McmaxClaire', 'MeliSa48999225',
        'Msblowana', 'MyxaZZ', 'NSolomonfair', 'Nehanotknown',
        'PassionateNana', 'PerryAlexJ', 'Qtherise', 'QueenRania',
        'RebelXentric', 'RepKenBuck', 'SharylAttkisson', 'SwineFluHeds',
        'SwineFluNewsIN', 'TannersDad', 'TexG11', 'TheFuknPunisher',
        'ThisBastard', 'Tishestrada1988', 'TransparentMed1', 'TryingWake',
        'Tuenchai33', 'Vinod65001857'

In [8]:
[half_dozen_dict[x].size for x in half_dozen]

[112, 105, 8, 26, 38, 22]

In [9]:
users_df = pd.read_csv("disinfo12_replies.csv")

In [10]:
error_df = pd.read_csv("disinfo12_errors.csv")

In [11]:
users_df

,Unnamed: 0,username,name,verified,id,description,protected,location,created_at,profile_image_url,followers_count,following_count,tweet_count,listed_count,url
0,0,1ThessCh5,Sarah Grace,False,1241472932456599552,"JESUS IS KING OF KINGS, AND LORD OF LORDS 🥰",False,"Black Mountain, NC",2020-03-21T21:13:16.000Z,https://pbs.twimg.com/profile_images/155307889...,24905,21376,38458,26,NaN
1,1,2600hz,2600hz 🇧🇷🇧🇷🇧🇷,False,73229721,"“Rage, rage against the dying of the light of ...",False,Brasil,2009-09-10T21:47:26.000Z,https://pbs.twimg.com/profile_images/124863890...,917,1649,2945,19,NaN
2,2,7dancingrain7,.....,False,823194601368461313,NaN,False,NaN,2017-01-22T15:44:22.000Z,https://pbs.twimg.com/profile_images/125157173...,6,39,1104,0,NaN
3,3,AldnoahVoid,Rafi Ali,False,24541819,"Talk sports, the truth & whatever I feel like....",False,"Gotham, NY",2009-03-15T16:02:56.000Z,https://pbs.twimg.com/profile_images/147824155...,184,772,16594,4,NaN
4,4,AlecBaldwin,AlecBaldwin(HABF),True,288452305,Hilaria and Alec Baldwin Foundation. Supportin...,True,"New York, NY",2011-04-26T21:45:30.000Z,https://pbs.twimg.com/profile_images/147415903...,1032141,2101,29982,8006,https://t.co/RyEIVjgbRQ
5,5,AlexBerenson,Alex Berenson,True,87679233,Author of Tell Your Children and Pandemia. Bac...,False,New York,2009-11-05T12:58:35.000Z,https://pbs.twimg.com/profile_images/128821533...,425653,208,19700,2571,https://t.co/B75By8xZ7F
6,6,AlisonRoseLevy,Alison Rose Levy,False,107114920,"Connecting the dots on health, food, environme...",False,"New York, NY",2010-01-21T16:00:26.000Z,https://pbs.twimg.com/profile_images/646237650...,1634,1589,10614,126,http://t.co/HWpcQzvMqE
7,7,AncestorAligned,𝒜𝓉𝓊𝓂 🕊,False,509687746,Truth is beauty and I am art • feminine not fe...,False,The land of milk and honey,2012-03-01T03:47:46.000Z,https://pbs.twimg.com/profile_images/158289502...,10988,997,72723,43,NaN
8,8,AndrewRamosTV,Andrew Ramos,True,33768456,People tell me their stories. I put it on TV. ...,False,NY | NJ,2009-04-21T02:06:33.000Z,https://pbs.twimg.com/profile_images/124568562...,3731,854,6290,72,https://t.co/aFhm4VA7Pl
9,9,AndrewSaulPhD,Andrew Saul PhD,False,1451811967,"Andrew Saul, Ph.D, Therapeutic Nutrition Speci...",False,"Rochester, New York",2013-05-23T15:37:10.000Z,https://pbs.twimg.com/profile_images/378800000...,2063,18,126,36,http://t.co/0qpzGUW2LR


In [12]:
error_df

,Unnamed: 0,parameter,resource_id,value,detail,title,resource_type,type
0,0,usernames,12Tommygun,12Tommygun,User has been suspended: [12Tommygun].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
1,1,usernames,8q4nx8bj,8q4nx8bj,User has been suspended: [8q4nx8bj].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
2,2,usernames,Achilles12219,Achilles12219,User has been suspended: [Achilles12219].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
3,3,usernames,BayouGirl54,BayouGirl54,User has been suspended: [BayouGirl54].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
4,4,usernames,BelleElle82,BelleElle82,User has been suspended: [BelleElle82].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
5,5,usernames,BusyDrT,BusyDrT,User has been suspended: [BusyDrT].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
6,6,usernames,CJforeveryoung,CJforeveryoung,User has been suspended: [CJforeveryoung].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
7,7,usernames,DollarrrrJim,DollarrrrJim,User has been suspended: [DollarrrrJim].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
8,8,usernames,EpigeneticWhisp,EpigeneticWhisp,User has been suspended: [EpigeneticWhisp].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...
9,9,usernames,ExpectingRain_,ExpectingRain_,User has been suspended: [ExpectingRain_].,Forbidden,user,https://api.twitter.com/2/problems/resource-no...


In [13]:
def common_members(a, b, half_dozen_dict):
    a_list = half_dozen_dict[a]
    b_list = half_dozen_dict[b]
    
    a_set = set(a_list)
    b_set = set(b_list)
 
    if (a_set & b_set):
        print (f'Common between {a}({i}) and {b}({j})')
        print(a_set & b_set)
    else:
        print("No common elements")

In [14]:
hd_keys = list(half_dozen_dict.keys())
hd_keys.sort()
for i in range(6):
    for j in range(i+1,6):
        common_members(hd_keys[i], hd_keys[j], half_dozen_dict)

Common between DrButtar(0) and DrChrisNorthrup(1)
{'realDonaldTrump'}
Common between DrButtar(0) and RobertKennedyJr(2)
{'realDonaldTrump'}
No common elements
No common elements
No common elements
Common between DrChrisNorthrup(1) and RobertKennedyJr(2)
{'ChildrensHD', 'realDonaldTrump'}
Common between DrChrisNorthrup(1) and kevdjenkins1(3)
{'eileeniorio'}
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
Common between mercola(4) and sayerjigmi(5)
{'CCDHate'}


In [15]:
hd_keys

['DrButtar',
 'DrChrisNorthrup',
 'RobertKennedyJr',
 'kevdjenkins1',
 'mercola',
 'sayerjigmi']

In [16]:
nodes_df = users_df[['username', 'name', 'verified', 'description', 'protected', 'location', 'created_at', 'followers_count', 'following_count', 'tweet_count', 'listed_count']]

In [17]:
for i in range(0,6):
    dude_list = half_dozen_dict[hd_keys[i]]
    for dude in dude_list:
        nodes_df.loc[nodes_df['username'] == dude, 'group'] = int(i+1);

/Users/iankelk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/iankelk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### For the few users being messaged by multiple disinfo people, set the groups to special ones. Also set a special group to add in suspended users.

In [18]:
nodes_df.loc[nodes_df['username'] == 'ChildrensHD','group'] = int(7)
nodes_df.loc[nodes_df['username'] == 'eileeniorio','group'] = int(8)
nodes_df.loc[nodes_df['username'] == 'CCDHate','group'] = int(9)
nodes_df.loc[nodes_df['username'] == 'realDonaldTrump','group'] = int(10)

In [19]:
nodes_df

,username,name,verified,description,protected,location,created_at,followers_count,following_count,tweet_count,listed_count,group
0,1ThessCh5,Sarah Grace,False,"JESUS IS KING OF KINGS, AND LORD OF LORDS 🥰",False,"Black Mountain, NC",2020-03-21T21:13:16.000Z,24905,21376,38458,26,2.0
1,2600hz,2600hz 🇧🇷🇧🇷🇧🇷,False,"“Rage, rage against the dying of the light of ...",False,Brasil,2009-09-10T21:47:26.000Z,917,1649,2945,19,1.0
2,7dancingrain7,.....,False,NaN,False,NaN,2017-01-22T15:44:22.000Z,6,39,1104,0,1.0
3,AldnoahVoid,Rafi Ali,False,"Talk sports, the truth & whatever I feel like....",False,"Gotham, NY",2009-03-15T16:02:56.000Z,184,772,16594,4,1.0
4,AlecBaldwin,AlecBaldwin(HABF),True,Hilaria and Alec Baldwin Foundation. Supportin...,True,"New York, NY",2011-04-26T21:45:30.000Z,1032141,2101,29982,8006,3.0
5,AlexBerenson,Alex Berenson,True,Author of Tell Your Children and Pandemia. Bac...,False,New York,2009-11-05T12:58:35.000Z,425653,208,19700,2571,5.0
6,AlisonRoseLevy,Alison Rose Levy,False,"Connecting the dots on health, food, environme...",False,"New York, NY",2010-01-21T16:00:26.000Z,1634,1589,10614,126,6.0
7,AncestorAligned,𝒜𝓉𝓊𝓂 🕊,False,Truth is beauty and I am art • feminine not fe...,False,The land of milk and honey,2012-03-01T03:47:46.000Z,10988,997,72723,43,2.0
8,AndrewRamosTV,Andrew Ramos,True,People tell me their stories. I put it on TV. ...,False,NY | NJ,2009-04-21T02:06:33.000Z,3731,854,6290,72,4.0
9,AndrewSaulPhD,Andrew Saul PhD,False,"Andrew Saul, Ph.D, Therapeutic Nutrition Speci...",False,"Rochester, New York",2013-05-23T15:37:10.000Z,2063,18,126,36,6.0


In [20]:
hd_followers = [87813, 114347, 483114, 16541, 361875, 11876]
hd_verified = [False, False, True, False, False, False]
hd_group = [12,12,12,12,12,12]               
misinfo6_df = pd.DataFrame({"id": hd_keys, "followers_count": hd_followers, "verified": hd_verified, "group": hd_group})

In [21]:
misinfo6_df.to_json("misinfo6-people.json", orient='records')
misinfo6_df

,id,followers_count,verified,group
0,DrButtar,87813,False,12
1,DrChrisNorthrup,114347,False,12
2,RobertKennedyJr,483114,True,12
3,kevdjenkins1,16541,False,12
4,mercola,361875,False,12
5,sayerjigmi,11876,False,12


In [22]:
nodes_err_df = error_df[['resource_id']]
nodes_err_df.columns = [['username']]

In [23]:
# nodes_err_df[['followers_count', 'verified', 'group']] = [0,False,11]


nodes_err_df[['name', 'verified', 'description',
              'protected', 'location', 'created_at', 'followers_count', 
              'following_count', 'tweet_count', 'listed_count', 'group']] = ["Unknown",False,error_df['detail'],
                                                                    False, "Unknown", "", 0,
                                                                    0,0,0,11]

/Users/iankelk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/Users/iankelk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [24]:
nodes_err_df.columns = ['username', 'name', 'verified', 'description', 'protected', 'location', 'created_at', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'group']
nodes_err_df

,username,name,verified,description,protected,location,created_at,followers_count,following_count,tweet_count,listed_count,group
0,12Tommygun,Unknown,False,User has been suspended: [12Tommygun].,False,Unknown,,0,0,0,0,11
1,8q4nx8bj,Unknown,False,User has been suspended: [8q4nx8bj].,False,Unknown,,0,0,0,0,11
2,Achilles12219,Unknown,False,User has been suspended: [Achilles12219].,False,Unknown,,0,0,0,0,11
3,BayouGirl54,Unknown,False,User has been suspended: [BayouGirl54].,False,Unknown,,0,0,0,0,11
4,BelleElle82,Unknown,False,User has been suspended: [BelleElle82].,False,Unknown,,0,0,0,0,11
5,BusyDrT,Unknown,False,User has been suspended: [BusyDrT].,False,Unknown,,0,0,0,0,11
6,CJforeveryoung,Unknown,False,User has been suspended: [CJforeveryoung].,False,Unknown,,0,0,0,0,11
7,DollarrrrJim,Unknown,False,User has been suspended: [DollarrrrJim].,False,Unknown,,0,0,0,0,11
8,EpigeneticWhisp,Unknown,False,User has been suspended: [EpigeneticWhisp].,False,Unknown,,0,0,0,0,11
9,ExpectingRain_,Unknown,False,User has been suspended: [ExpectingRain_].,False,Unknown,,0,0,0,0,11


In [25]:
nodes_df

,username,name,verified,description,protected,location,created_at,followers_count,following_count,tweet_count,listed_count,group
0,1ThessCh5,Sarah Grace,False,"JESUS IS KING OF KINGS, AND LORD OF LORDS 🥰",False,"Black Mountain, NC",2020-03-21T21:13:16.000Z,24905,21376,38458,26,2.0
1,2600hz,2600hz 🇧🇷🇧🇷🇧🇷,False,"“Rage, rage against the dying of the light of ...",False,Brasil,2009-09-10T21:47:26.000Z,917,1649,2945,19,1.0
2,7dancingrain7,.....,False,NaN,False,NaN,2017-01-22T15:44:22.000Z,6,39,1104,0,1.0
3,AldnoahVoid,Rafi Ali,False,"Talk sports, the truth & whatever I feel like....",False,"Gotham, NY",2009-03-15T16:02:56.000Z,184,772,16594,4,1.0
4,AlecBaldwin,AlecBaldwin(HABF),True,Hilaria and Alec Baldwin Foundation. Supportin...,True,"New York, NY",2011-04-26T21:45:30.000Z,1032141,2101,29982,8006,3.0
5,AlexBerenson,Alex Berenson,True,Author of Tell Your Children and Pandemia. Bac...,False,New York,2009-11-05T12:58:35.000Z,425653,208,19700,2571,5.0
6,AlisonRoseLevy,Alison Rose Levy,False,"Connecting the dots on health, food, environme...",False,"New York, NY",2010-01-21T16:00:26.000Z,1634,1589,10614,126,6.0
7,AncestorAligned,𝒜𝓉𝓊𝓂 🕊,False,Truth is beauty and I am art • feminine not fe...,False,The land of milk and honey,2012-03-01T03:47:46.000Z,10988,997,72723,43,2.0
8,AndrewRamosTV,Andrew Ramos,True,People tell me their stories. I put it on TV. ...,False,NY | NJ,2009-04-21T02:06:33.000Z,3731,854,6290,72,4.0
9,AndrewSaulPhD,Andrew Saul PhD,False,"Andrew Saul, Ph.D, Therapeutic Nutrition Speci...",False,"Rochester, New York",2013-05-23T15:37:10.000Z,2063,18,126,36,6.0


In [26]:
# nodes_df.columns = ['id','followers_count', 'verified', 'group']
# nodes_err_df.columns = ['id','followers_count', 'verified', 'group']
all_nodes_df = pd.concat([nodes_df,nodes_err_df], ignore_index=True)

In [27]:
all_nodes_df

,username,name,verified,description,protected,location,created_at,followers_count,following_count,tweet_count,listed_count,group
0,1ThessCh5,Sarah Grace,False,"JESUS IS KING OF KINGS, AND LORD OF LORDS 🥰",False,"Black Mountain, NC",2020-03-21T21:13:16.000Z,24905,21376,38458,26,2.0
1,2600hz,2600hz 🇧🇷🇧🇷🇧🇷,False,"“Rage, rage against the dying of the light of ...",False,Brasil,2009-09-10T21:47:26.000Z,917,1649,2945,19,1.0
2,7dancingrain7,.....,False,NaN,False,NaN,2017-01-22T15:44:22.000Z,6,39,1104,0,1.0
3,AldnoahVoid,Rafi Ali,False,"Talk sports, the truth & whatever I feel like....",False,"Gotham, NY",2009-03-15T16:02:56.000Z,184,772,16594,4,1.0
4,AlecBaldwin,AlecBaldwin(HABF),True,Hilaria and Alec Baldwin Foundation. Supportin...,True,"New York, NY",2011-04-26T21:45:30.000Z,1032141,2101,29982,8006,3.0
5,AlexBerenson,Alex Berenson,True,Author of Tell Your Children and Pandemia. Bac...,False,New York,2009-11-05T12:58:35.000Z,425653,208,19700,2571,5.0
6,AlisonRoseLevy,Alison Rose Levy,False,"Connecting the dots on health, food, environme...",False,"New York, NY",2010-01-21T16:00:26.000Z,1634,1589,10614,126,6.0
7,AncestorAligned,𝒜𝓉𝓊𝓂 🕊,False,Truth is beauty and I am art • feminine not fe...,False,The land of milk and honey,2012-03-01T03:47:46.000Z,10988,997,72723,43,2.0
8,AndrewRamosTV,Andrew Ramos,True,People tell me their stories. I put it on TV. ...,False,NY | NJ,2009-04-21T02:06:33.000Z,3731,854,6290,72,4.0
9,AndrewSaulPhD,Andrew Saul PhD,False,"Andrew Saul, Ph.D, Therapeutic Nutrition Speci...",False,"Rochester, New York",2013-05-23T15:37:10.000Z,2063,18,126,36,6.0


In [28]:
all_nodes_df['group'] = all_nodes_df['group'].astype(int)
all_nodes_df.to_json("nodes.json", orient='records')
all_nodes_df

,username,name,verified,description,protected,location,created_at,followers_count,following_count,tweet_count,listed_count,group
0,1ThessCh5,Sarah Grace,False,"JESUS IS KING OF KINGS, AND LORD OF LORDS 🥰",False,"Black Mountain, NC",2020-03-21T21:13:16.000Z,24905,21376,38458,26,2
1,2600hz,2600hz 🇧🇷🇧🇷🇧🇷,False,"“Rage, rage against the dying of the light of ...",False,Brasil,2009-09-10T21:47:26.000Z,917,1649,2945,19,1
2,7dancingrain7,.....,False,NaN,False,NaN,2017-01-22T15:44:22.000Z,6,39,1104,0,1
3,AldnoahVoid,Rafi Ali,False,"Talk sports, the truth & whatever I feel like....",False,"Gotham, NY",2009-03-15T16:02:56.000Z,184,772,16594,4,1
4,AlecBaldwin,AlecBaldwin(HABF),True,Hilaria and Alec Baldwin Foundation. Supportin...,True,"New York, NY",2011-04-26T21:45:30.000Z,1032141,2101,29982,8006,3
5,AlexBerenson,Alex Berenson,True,Author of Tell Your Children and Pandemia. Bac...,False,New York,2009-11-05T12:58:35.000Z,425653,208,19700,2571,5
6,AlisonRoseLevy,Alison Rose Levy,False,"Connecting the dots on health, food, environme...",False,"New York, NY",2010-01-21T16:00:26.000Z,1634,1589,10614,126,6
7,AncestorAligned,𝒜𝓉𝓊𝓂 🕊,False,Truth is beauty and I am art • feminine not fe...,False,The land of milk and honey,2012-03-01T03:47:46.000Z,10988,997,72723,43,2
8,AndrewRamosTV,Andrew Ramos,True,People tell me their stories. I put it on TV. ...,False,NY | NJ,2009-04-21T02:06:33.000Z,3731,854,6290,72,4
9,AndrewSaulPhD,Andrew Saul PhD,False,"Andrew Saul, Ph.D, Therapeutic Nutrition Speci...",False,"Rochester, New York",2013-05-23T15:37:10.000Z,2063,18,126,36,6


### A few things I do manually now
* Set realDonaldTrump to have 88.9 million followers as he did before being suspended
* Give realDonaldTrump a value of 10 to make the links bigger and arrows larger
* Remove duplicate Kennedy pointing to himself
* Manually adding DrChrisNorthrup to the nodes since he didn't appear as a recipient

### Now create the link JSON

In [29]:
import math

source_list = []
target_list = []
value_list = []
for i in range(0,6):
    source_dude = hd_keys[i]
    dude_list = half_dozen_dict[source_dude]
    for dude in dude_list:
        source_list = source_list + [source_dude]
        target_list = target_list + [dude]
        # Scale the followers logarithmically since they have a massive range from 0 to hundreds of millions
        followers_count = all_nodes_df.loc[all_nodes_df['username'] == dude]['followers_count'].iloc[0]+1
        value_list = value_list + [int(math.log10(followers_count)+1)]

In [30]:
links_df = pd.DataFrame({"source": source_list, "target": target_list, "value": value_list})

In [31]:
links_df.to_json("links.json", orient='records')

In [32]:
# pd.concat([all_nodes_df,links_df], ignore_index=True).to_json("misinfo6.json")

In [33]:
str(all_nodes_df['listed_count'].min()) + " " + str(all_nodes_df['listed_count'].max())

'0 216613'

In [34]:
df.sort_values(by=['created_at'])

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
7534,NaN,Fri Apr 01 10:05:02 +0000 2016,NaN,NaN,http://aweber.com/t/RSRhA,1,715842398937604096,NaN,NaN,NaN,en,NaN,False,NaN,1,NaN,NaN,"<a href=""http://www.aweber.com"" rel=""nofollow""...",H.O.W. Report Weekly Newsletter - - Dr. Rashi...,https://twitter.com/DrButtar/status/7158423989...,Mon Aug 10 15:03:02 +0000 2009,64423855,False,Our Mission:Making the Change the World is wai...,144,87689,28,285,"Charlotte, North Carolina, USA","Dr Rashid A Buttar, FAAPM, FACAM, FAAIM",DrButtar,3146,NaN,https://www.DrButtar.com,False
13713,NaN,Fri Apr 02 10:40:18 +0000 2021,NaN,NaN,https://mailchi.mp/aapsonline/thank-gov-desantis,0,1377933906473803780,NaN,NaN,NaN,en,NaN,False,NaN,65,1.377301e+18,DrChrisNorthrup,"<a href=""http://twitter.com/download/iphone"" r...","Thank Gov. DeSantis for saying ""no"" to vaccine...",https://twitter.com/kevdjenkins1/status/137793...,Sat Oct 17 19:50:10 +0000 2015,3981743673,False,"Husband, Father, Humanitarian, CEO and Producer",3064,15574,500,30,Patriot,Kevin Jenkins,kevdjenkins1,3632,NaN,http://zfreedomfoundation.org,False
9323,NaN,Fri Apr 02 12:31:15 +0000 2021,NaN,NaN,NaN,0,1377961828194643977,NaN,NaN,NaN,en,NaN,NaN,NaN,558,1.377274e+18,MaximeBernier,"<a href=""http://twitter.com/download/iphone"" r...",Politicians responsible for this are not just ...,https://twitter.com/sayerjigmi/status/13779618...,Sat Jun 15 12:47:17 +0000 2013,1519338979,False,"Sayer is an author, philosopher, activist, ent...",234,11888,1035,137,"Miami, FL",Sayer Ji,sayerjigmi,1622,NaN,https://withkoji.com/@sayerji,False
535,NaN,Fri Apr 02 13:01:19 +0000 2021,NaN,https://twitter.com/mercola/status/13779693930...,NaN,697,1377969393003601921,NaN,NaN,NaN,en,NaN,False,NaN,208,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...","You okay, @google? https://t.co/cgToYQMeoz",https://twitter.com/mercola/status/13779693930...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False
532,NaN,Fri Apr 02 13:19:32 +0000 2021,NaN,NaN,http://dlvr.it/Rwt45h,120,1377973976945680387,NaN,NaN,NaN,en,NaN,False,NaN,83,NaN,NaN,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",The power that Bill Gates wields as the bigges...,https://twitter.com/mercola/status/13779739769...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11618,NaN,Wed Sep 30 17:38:07 +0000 2020,NaN,NaN,https://youtu.be/hRrmTmtnAGY,9,1311359685560279047,NaN,NaN,NaN,en,NaN,False,NaN,1,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",I'm proud of this guy-- my nephew. Kirk Matthe...,https://twitter.com/DrChrisNorthrup/status/131...,Tue Feb 16 17:33:01 +0000 2010,114807372,False,Dr. Christiane Northrup is a leading authority...,75,114775,594,1704,Maine,Christiane Northrup,DrChrisNorthrup,12072,NaN,http://www.drnorthrup.com,False
1389,NaN,Wed Sep 30 17:45:09 +0000 2020,NaN,NaN,NaN,38,1311361456009555985,NaN,NaN,NaN,en,NaN,NaN,NaN,12,NaN,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...",The only element who cares in this whole globa...,https://twitter.com/mercola/status/13113614560...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic phy

In [35]:
from dateutil import parser
#yourdate = parser.parse(datestring)

#df['date'] = pd.to_datetime(df['created_at'])
#df['date'] = pd.to_datetime(df['created_at'])

df['date'] = df['created_at'].apply(parser.parse)

df = df.sort_values(by=['date'])

In [36]:
df

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,date
9283,NaN,Mon Aug 10 15:44:29 +0000 2009,NaN,NaN,NaN,0,3226850395,usmile777,3.207037e+09,16991479.0,qam,NaN,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@usmile777,https://twitter.com/DrButtar/status/3226850395,Mon Aug 10 15:03:02 +0000 2009,64423855,False,Our Mission:Making the Change the World is wai...,144,87689,28,285,"Charlotte, North Carolina, USA","Dr Rashid A Buttar, FAAPM, FACAM, FAAIM",DrButtar,3146,NaN,https://www.DrButtar.com,False,2009-08-10 15:44:29+00:00
9279,NaN,Mon Aug 10 15:50:04 +0000 2009,NaN,NaN,NaN,0,3226941967,NaN,NaN,NaN,en,NaN,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Hi...I just joined Twitter. Thank you for your...,https://twitter.com/DrButtar/status/3226941967,Mon Aug 10 15:03:02 +0000 2009,64423855,False,Our Mission:Making the Change the World is wai...,144,87689,28,285,"Charlotte, North Carolina, USA","Dr Rashid A Buttar, FAAPM, FACAM, FAAIM",DrButtar,3146,NaN,https://www.DrButtar.com,False,2009-08-10 15:50:04+00:00
9313,NaN,Tue Aug 11 03:46:29 +0000 2009,NaN,NaN,NaN,0,3239128278,NaN,NaN,NaN,en,NaN,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",48 hours to launch for FactsOnToxicity.com. 9 ...,https://twitter.com/DrButtar/status/3239128278,Mon Aug 10 15:03:02 +0000 2009,64423855,False,Our Mission:Making the Change the World is wai...,144,87689,28,285,"Charlotte, North Carolina, USA","Dr Rashid A Buttar, FAAPM, FACAM, FAAIM",DrButtar,3146,NaN,https://www.DrButtar.com,False,2009-08-11 03:46:29+00:00
9281,NaN,Fri Aug 21 22:26:09 +0000 2009,NaN,NaN,NaN,1,3459872358,NaN,NaN,NaN,en,NaN,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Learn about the True Cause of All Disease. All...,https://twitter.com/DrButtar/status/3459872358,Mon Aug 10 15:03:02 +0000 2009,64423855,False,Our Mission:Making the Change the World is wai...,144,87689,28,285,"Charlotte, North Carolina, USA","Dr Rashid A Buttar, FAAPM, FACAM, FAAIM",DrButtar,3146,NaN,https://www.DrButtar.com,False,2009-08-21 22:26:09+00:00
9293,NaN,Thu Oct 08 02:48:12 +0000 2009,NaN,NaN,NaN,0,4699553306,NaN,NaN,NaN,en,NaN,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Baxter got patent on H1N1 swine flu 2 years be...,https://twitter.com/DrButtar/status/4699553306,Mon Aug 10 15:03:02 +0000 2009,64423855,False,Our Mission:Making the Change the World is wai...,144,87689,28,285,"Charlotte, North Carolina, USA","Dr Rashid A Buttar, FAAPM, FACAM, FAAIM",DrButtar,3146,NaN,https://www.DrButtar.com,False,2009-10-08 02:48:12+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,NaN,Tue Aug 03 19:23:57 +0000 2021,NaN,NaN,https://www.foxnews.com/health/covid-nih-direc...,151,1422639392460521477,NaN,NaN,NaN,en,NaN,False,NaN,43,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",The eternal string of public health messaging ...,https://twitter.com/mercola/status/14226393924...,Tue Jan 22 05:37:28 +0000 2008,12524522,False,"Osteopathic physician, best-selling author, an...",1653,361271,28302,5714,"Cape Coral, FL",Dr. Joseph Mercola,mercola,81328,NaN,http://dlvr.it/S5SBZN,False,2021-08-03 19:23:57+00:00
46,NaN,Tue Aug 03 20:05:54 +0000 2021,NaN,NaN,https://www.youtube.com/watch?v=09maaUaRT4M,281,1422649949616017409,NaN,NaN,NaN,en,NaN,False,NaN,124,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Objects, Dictators, and Mass Psychos

In [54]:
# df['date'] = pd.date_range('2000-1-1', periods=15647, freq='D')
mask = (df['date'] > '2020-01-22') & (df['date'] <= '2022-10-17')
covid_range_df = df.loc[mask][['date','user_screen_name','favorite_count', 'retweet_count']]
covid_range_df = covid_range_df.reset_index()

In [49]:
covid_range_df.shape

(7119, 4)

In [50]:
covid_range_df.columns

Index(['date', 'user_screen_name', 'favorite_count', 'retweet_count'], dtype='object')

In [51]:
[['date','user_screen_name','favorite_count', 'retweet_count']]

[['date', 'user_screen_name', 'favorite_count', 'retweet_count']]

In [52]:
#df_vis = pd.DataFrame(df.groupby(['date', pd.Grouper(key='Publication_Date', freq='W-MON')]).count())
#df_vis = df_vis.reset_index()


In [82]:
covid_range_df['date'] = pd.to_datetime(covid_range_df['date']).dt.date
covid_range_df['date_copy_to_count'] = covid_range_df['date']
covid_range_df.head(50)

,index,date,user_screen_name,favorite_count,retweet_count,date_copy_to_count
0,2695,2020-01-22,mercola,5,5,2020-01-22
1,4352,2020-01-22,RobertKennedyJr,30,9,2020-01-22
2,2630,2020-01-22,mercola,5,3,2020-01-22
3,2643,2020-01-22,mercola,27,10,2020-01-22
4,14411,2020-01-22,kevdjenkins1,0,110,2020-01-22
5,2637,2020-01-22,mercola,56,31,2020-01-22
6,4301,2020-01-22,RobertKennedyJr,146,88,2020-01-22
7,2680,2020-01-22,mercola,13,10,2020-01-22
8,2678,2020-01-22,mercola,12,13,2020-01-22
9,4346,2020-01-22,RobertKennedyJr,217,64,2020-01-22


In [83]:
df_vis = (covid_range_df.groupby(["date", "user_screen_name"])
 .agg({'favorite_count':'sum', 'retweet_count': 'sum', 'date_copy_to_count': 'count'})
 .reset_index()
 .rename(columns={'favorite_count':'favorites', 'retweet_count':'retweets', 'date_copy_to_count':'tweets'}))

In [84]:
df_vis.head(20)

,date,user_screen_name,favorites,retweets,tweets
0,2020-01-22,DrChrisNorthrup,15,90,4
1,2020-01-22,RobertKennedyJr,393,169,4
2,2020-01-22,kevdjenkins1,0,110,1
3,2020-01-22,mercola,191,102,7
4,2020-01-23,DrChrisNorthrup,10,4,1
5,2020-01-23,RobertKennedyJr,262,227,4
6,2020-01-23,kevdjenkins1,1,17,2
7,2020-01-23,mercola,255,186,13
8,2020-01-24,DrChrisNorthrup,22,9,2
9,2020-01-24,RobertKennedyJr,213,104,3


In [86]:
df_vis.to_json("misinfo6_tweets.json", orient='records', date_format='iso')

In [88]:
df_vis.head()

,date,user_screen_name,favorites,retweets,tweets
0,2020-01-22,DrChrisNorthrup,15,90,4
1,2020-01-22,RobertKennedyJr,393,169,4
2,2020-01-22,kevdjenkins1,0,110,1
3,2020-01-22,mercola,191,102,7
4,2020-01-23,DrChrisNorthrup,10,4,1


In [ ]:
for dude in half_dozen:
    dude_recipients = df.loc[df['user_screen_name'] == dude]
    dude_recipients = dude_recipients['in_reply_to_screen_name'].unique()[1:]
    dude_recipients.sort()
    half_dozen_dict[dude] = dude_recipients

array(['DrButtar', 'DrChrisNorthrup', 'RobertKennedyJr', 'kevdjenkins1',
       'mercola', 'sayerjigmi'], dtype=object)

In [104]:
df_vis.columns = ['date', 'username', 'favorites', 'retweets', 'tweets']
df_vis

,date,username,favorites,retweets,tweets
0,2020-01-22,DrChrisNorthrup,15,90,4
1,2020-01-22,RobertKennedyJr,393,169,4
2,2020-01-22,kevdjenkins1,0,110,1
3,2020-01-22,mercola,191,102,7
4,2020-01-23,DrChrisNorthrup,10,4,1
...,...,...,...,...,...
2035,2021-08-03,kevdjenkins1,0,132,3
2036,2021-08-03,mercola,9238,2919,16
2037,2021-08-04,RobertKennedyJr,1748,908,1
2038,2021-08-04,kevdjenkins1,0,1679,1


In [106]:
df_favorites = pd.pivot(df_vis, index='date', columns='username', values='favorites').fillna(0).reset_index()
df_favorites.to_json("favorites.json", orient='records', date_format='iso')
df_favorites

username,date,DrButtar,DrChrisNorthrup,RobertKennedyJr,kevdjenkins1,mercola,sayerjigmi
0,2020-01-22,0.0,15.0,393.0,0.0,191.0,0.0
1,2020-01-23,0.0,10.0,262.0,1.0,255.0,0.0
2,2020-01-24,0.0,22.0,213.0,0.0,203.0,0.0
3,2020-01-25,0.0,0.0,198.0,1.0,156.0,0.0
4,2020-01-26,0.0,0.0,166.0,0.0,205.0,0.0
...,...,...,...,...,...,...,...
555,2021-07-31,0.0,7.0,0.0,0.0,0.0,0.0
556,2021-08-01,0.0,51.0,0.0,0.0,0.0,0.0
557,2021-08-02,0.0,0.0,5599.0,0.0,4504.0,0.0
558,2021-08-03,0.0,6.0,2594.0,0.0,9238.0,0.0


In [107]:
df_retweets = pd.pivot(df_vis, index='date', columns='username', values='retweets').fillna(0).reset_index()
df_retweets.to_json("retweets.json", orient='records', date_format='iso')
df_retweets

username,date,DrButtar,DrChrisNorthrup,RobertKennedyJr,kevdjenkins1,mercola,sayerjigmi
0,2020-01-22,0.0,90.0,169.0,110.0,102.0,0.0
1,2020-01-23,0.0,4.0,227.0,17.0,186.0,0.0
2,2020-01-24,0.0,9.0,104.0,0.0,133.0,0.0
3,2020-01-25,0.0,0.0,107.0,0.0,61.0,0.0
4,2020-01-26,0.0,0.0,87.0,0.0,117.0,0.0
...,...,...,...,...,...,...,...
555,2021-07-31,0.0,1.0,0.0,0.0,0.0,0.0
556,2021-08-01,0.0,11.0,0.0,0.0,0.0,0.0
557,2021-08-02,0.0,0.0,2148.0,0.0,1370.0,0.0
558,2021-08-03,0.0,0.0,1315.0,132.0,2919.0,0.0


In [108]:
df_tweets = pd.pivot(df_vis, index='date', columns='username', values='tweets').fillna(0).reset_index()
df_tweets.to_json("tweets.json", orient='records', date_format='iso')
df_tweets

username,date,DrButtar,DrChrisNorthrup,RobertKennedyJr,kevdjenkins1,mercola,sayerjigmi
0,2020-01-22,0.0,4.0,4.0,1.0,7.0,0.0
1,2020-01-23,0.0,1.0,4.0,2.0,13.0,0.0
2,2020-01-24,0.0,2.0,3.0,0.0,9.0,0.0
3,2020-01-25,1.0,0.0,2.0,1.0,7.0,0.0
4,2020-01-26,0.0,0.0,2.0,0.0,7.0,0.0
...,...,...,...,...,...,...,...
555,2021-07-31,0.0,1.0,0.0,0.0,0.0,0.0
556,2021-08-01,0.0,1.0,0.0,0.0,0.0,0.0
557,2021-08-02,0.0,0.0,2.0,0.0,15.0,0.0
558,2021-08-03,0.0,1.0,2.0,3.0,16.0,0.0
